In [1]:
import pandas as pd
import numpy as np
population_df = pd.read_csv("../data/synthetic_population.csv")

In [4]:
def initialize_individual_risks(df):
    """
    Maps demographic features to individual SEIR parameters.
    """ 
    df = df.copy()

    df["transmission_multiplier"] = 1.0

    df['transmission_multiplier'] = np.where(df['frontline_worker'] == 1, df['transmission_multiplier'] * 1.8, df['transmission_multiplier'])

    df['transmission_multiplier'] = np.where(df['multigenerational'] == 1, df['transmission_multiplier'] * 1.3, df['transmission_multiplier'])


    # SES Quintile
    ses_map = {1: 1.5, 2: 1.3, 3: 1.1, 4: 1.0, 5: 0.8}
    df['transmission_multiplier'] *= df['ses_quintile'].map(ses_map).fillna(1.0)

    # Mortality Risk
    # Risk increases exponentially with age
    # Base age 25, doubling risk every ~8 years
    df['age_factor'] = np.exp(0.09 * (df['age'] - 25))

    # Comorbidity Odds Ratio (estimated from CDC/NIH data)
    comorbidity_weights = {
        'has_diabetes': 1.9,
        'has_hypertension': 1.5,
        'has_heart_disease': 2.1,
        'has_copd_or_chronic_resp_disease': 2.0,
        'has_obesity': 1.6,
        'is_long_term_care': 5.0  # Huge risk factor
    }

    df['comorbidity_multiplier'] = 1.0
    for feature, weight in comorbidity_weights.items():
        df.loc[df[feature] == 1, 'comorbidity_multiplier'] *= weight
    

    # Final individual mortality Probability
    base_ifr = 0.005 #0.5% baseline
    df['individual_mortality_prob'] = base_ifr * df['age_factor'] * df['comorbidity_multiplier']

    # Clip probabilities at 1.0
    df['individual_mortality_prob'] = df['individual_mortality_prob'].clip(0, 1)

    return df

def run_agent_based_seir(df, days=100, initial_infected=10, base_beta=0.3):
    N = len(df)
    states = np.zeros(N) # 0:S, 1:E, 2:I, 3:R, 4:D
    
    # Seed infections
    states[np.random.choice(N, initial_infected, replace=False)] = 2
    
    # Pre-extract probabilities as arrays for speed
    trans_mult = df['transmission_multiplier'].values
    death_probs = df['individual_mortality_prob'].values
    
    sigma = 1/5.2  # E -> I rate
    gamma = 1/10.0 # I -> R/D rate
    history = []

    for t in range(days):
        infected_mask = (states == 2)
        n_infected = np.sum(infected_mask)
        
        history.append({'S': np.sum(states==0), 'E': np.sum(states==1), 
                        'I': n_infected, 'R': np.sum(states==3), 'D': np.sum(states==4)})

        if n_infected == 0: break

        # Calculate current infection risk for everyone
        force_of_infection = base_beta * (n_infected / N)
        
        # 1. Transitions: S -> E
        s_mask = (states == 0)
        # Everyone rolls the dice at once
        infect_roll = np.random.rand(N) < (force_of_infection * trans_mult)
        states[s_mask & infect_roll] = 1
        
        # 2. Transitions: E -> I
        e_mask = (states == 1)
        exposed_roll = np.random.rand(N) < sigma
        states[e_mask & exposed_roll] = 2
        
        # 3. Transitions: I -> R or D
        # Only check people who are currently infected and ready to "resolve"
        resolve_roll = np.random.rand(N) < gamma
        to_resolve = infected_mask & resolve_roll
        
        if np.any(to_resolve):
            # For those resolving, roll for death vs recovery
            death_roll = np.random.rand(N) < death_probs
            states[to_resolve & death_roll] = 4 # Dead
            states[to_resolve & ~death_roll] = 3 # Recovered

    return pd.DataFrame(history), states

In [5]:
# Execution
population_df = initialize_individual_risks(population_df)
sim_history, final_states = run_agent_based_seir(population_df)

In [6]:
# Create a summary of who died
population_df['is_dead'] = (final_states == 4).astype(int)

# Insight A: Mortality by Age
age_impact = population_df.groupby(pd.cut(population_df['age'], [0, 50, 65, 80, 110]))['is_dead'].mean()

# Insight B: Infection by Occupation
# (We need to see who entered the 'E', 'I', 'R', or 'D' states)
population_df['was_infected'] = (final_states > 0).astype(int)
job_impact = population_df.groupby('frontline_worker')['was_infected'].mean()

print(f"Mortality by Age:\n{age_impact}")
print(f"Infection Rate (Frontline vs Not):\n{job_impact}")

Mortality by Age:
age
(0, 50]      0.018418
(50, 65]     0.148691
(65, 80]     0.705664
(80, 110]    0.943803
Name: is_dead, dtype: float64
Infection Rate (Frontline vs Not):
frontline_worker
0    0.959607
1    0.996863
Name: was_infected, dtype: float64


C:\Users\vinee\AppData\Local\Temp\ipykernel_37528\3821457667.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_impact = population_df.groupby(pd.cut(population_df['age'], [0, 50, 65, 80, 110]))['is_dead'].mean()
